In [2]:
##Prepare API data
from Binance import Binance
# doc reference https://docs.binance.us/#get-candlestick-data

binance = Binance(BASE_URL = 'https://api.binance.us')
url_path = '/api/v3/klines'
payload = {
    'symbol': 'BTCUSDT',
    'interval': '1d',
    'startTime': 1640995200000,  # January 1st, 2021 00:00:00 in UTC
    'endTime': 1672444800000   # December 31st, 2021 23:59:59 in UTC
}

r = binance.send_public_request(url_path, payload=payload)


from pandas import json_normalize
import pandas as pd
col = ['Open time','Open','High','Low','Close','Volume','Close time','Quote asset volume','Number of trades',
       'Taker buy volume','Taker buy quote asset volume','Ignore']
print(pd.DataFrame(r, columns = col))

         Open time            Open            High             Low  \
0    1640995200000  46192.43000000  47939.39000000  46192.43000000   
1    1641081600000  47716.44000000  47975.79000000  46675.97000000   
2    1641168000000  47300.67000000  47573.70000000  45688.53000000   
3    1641254400000  46494.93000000  47535.30000000  45518.00000000   
4    1641340800000  45838.76000000  47056.53000000  42504.51000000   
..             ...             ...             ...             ...   
360  1672099200000  16921.80000000  16966.97000000  16604.33000000   
361  1672185600000  16707.35000000  16783.90000000  16471.43000000   
362  1672272000000  16547.63000000  16662.69000000  16492.62000000   
363  1672358400000  16631.15000000  16650.64000000  16369.84000000   
364  1672444800000  16605.84000000  16639.73000000  16466.79000000   

              Close         Volume     Close time Quote asset volume  \
0    47737.60000000   108.92603000  1641081599999   5130033.75669766   
1    47339.1500

In [3]:
##Set the data
data = pd.DataFrame(r, columns = col)
print(data)


         Open time            Open            High             Low  \
0    1640995200000  46192.43000000  47939.39000000  46192.43000000   
1    1641081600000  47716.44000000  47975.79000000  46675.97000000   
2    1641168000000  47300.67000000  47573.70000000  45688.53000000   
3    1641254400000  46494.93000000  47535.30000000  45518.00000000   
4    1641340800000  45838.76000000  47056.53000000  42504.51000000   
..             ...             ...             ...             ...   
360  1672099200000  16921.80000000  16966.97000000  16604.33000000   
361  1672185600000  16707.35000000  16783.90000000  16471.43000000   
362  1672272000000  16547.63000000  16662.69000000  16492.62000000   
363  1672358400000  16631.15000000  16650.64000000  16369.84000000   
364  1672444800000  16605.84000000  16639.73000000  16466.79000000   

              Close         Volume     Close time Quote asset volume  \
0    47737.60000000   108.92603000  1641081599999   5130033.75669766   
1    47339.1500

In [4]:
## Prepare Indicators data
## Common Periods: EMA 12 26 10 20 50 100 200
## Common scale: 10-200
import numpy
import talib

In [5]:
## EMA
## example: ema12 = talib.EMA(data['Close'], timeperiod=12)
## MA
## real = MA(close, timeperiod=30, matype=0)
## MIDPRICE - Midprice over period
## real = MIDPRICE(high,low, timeperiod=14)
## MIDPOINT - MidPoint over period
## real = MIDPOINT(close, timeperiod=14)
## WMA - Weighted Moving Average
## real = WMA(close, timeperiod=30)
for n in range(10,201):
    EMA = talib.EMA(data['Close'],timeperiod=n)
    MA = talib.MA(data['Close'], timeperiod=n)
    #WMA = talib.WMA(data["Close"], timeperiod=n)
    MIDPOINT = talib.MIDPOINT(data["Close"], timeperiod=n)
    data["MA_"+str(n)] = MA
    data["EMA_"+str(n)] = EMA
    #data["WMA_"+str(n)] = WMA
    data["MIDPOINT_"+str(n)] = MIDPOINT

WMA = talib.WMA(data["Close"], timeperiod=30)
data["WMA_"+str(30)] = WMA

MIDPRICE = talib.MIDPRICE(data["High"], data["Low"], timeperiod=14)
data["MIDPRICE_"+str(14)] = MIDPRICE





cols1 = [data["MA_"+str(n)] for n in range(10,201)] + [data["EMA_"+str(n)] for n in range(10,201)] + [
    data["WMA_"+str(30)]] + [data["MIDPRICE_"+str(14)]] + [data["MIDPOINT_"+str(n)] for n in range(10,201)]




/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/3337681096.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["MIDPOINT_"+str(n)] = MIDPOINT
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/3337681096.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["MA_"+str(n)] = MA
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/3337681096.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

In [6]:
# Momentum Indicator Functions
## ADX - Average Directional Movement Index
## real = ADX(high, low, close, timeperiod=14)
ADX = talib.ADX(data["High"], data["Low"],data["Close"], timeperiod=14)
data["ADX_"+str(14)] = ADX

## ADXR - Average Directional Movement Index Rating
## real = ADX(high, low, close, timeperiod=14)
ADXR = talib.ADXR(data["High"], data["Low"], data["Close"], timeperiod=14)
data["ADXR_"+str(14)] = ADXR

##APO - Absolute Price Oscillator
##real = APO(close, fastperiod=12, slowperiod=26, matype=0)
APO = talib.APO(data["Close"], fastperiod=12, slowperiod=26, matype=0)
data["APO"] = APO

## RSI - Relative Strength Index
## Note: The RSI function has an unstable period.
## real = RSI(close, timeperiod=14)
for n in range(10,201):
    RSI = talib.RSI(data["Close"], timeperiod=n)
    data["RSI_"+str(n)] = RSI

## ROC - Rate of change : ((price/prevPrice)-1)*100
## real = ROC(close, timeperiod=10)
for n in range(10,201):
    ROC = talib.ROC(data["Close"], timeperiod=10)
    data["ROC_"+str(n)] = ROC

## WILLR - Williams' %R
## real = WILLR(high, low, close, timeperiod=14)
for n in range(10,201):
    WILLR = talib.WILLR(data["High"], data["Low"], data["Close"], timeperiod=n)
    data["WILLR_"+str(n)] = WILLR

## ULTOSC - Ultimate Oscillator
## real = ULTOSC(high, low, close, timeperiod1=7, timeperiod2=14, timeperiod3=28)
ULTOSC = talib.ULTOSC(data["High"], data["Low"], data["Close"], timeperiod1=7, timeperiod2=14, timeperiod3=28)
data["ULTOSC"] = ULTOSC

## MOM - Momentum
## real = MOM(close, timeperiod=10)
for n in range(10,201):
    MOM = talib.MOM(data["Close"], timeperiod=n)
    data["MOM_"+str(n)] = MOM

## BOP - Balance Of Power
## real = BOP(open, high, low, close)
BOP = talib.BOP(data["Open"], data["High"], data["Low"], data["Close"])
data["BOP"] = BOP


cols2 = [data["ADX_"+str(14)]] + [data["ADXR_"+str(14)]] + [data["APO"]] + [data["RSI_"+str(n)] for n in range(10,201)] + [
    data["ROC_"+str(n)] for n in range(10,201)] + [data["WILLR_"+str(n)] for n in range(10,201)] + [data["ULTOSC"]] + [
    data["MOM_"+str(n)] for n in range(10,201)] + [data["BOP"]]






/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/3723665656.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["ADX_"+str(14)] = ADX
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/3723665656.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["ADXR_"+str(14)] = ADXR
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/3723665656.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfo

In [7]:
## Volume Indicator
##AD - Chaikin A/D Line
##real = AD(high, low, close, volume)
AD = talib.AD(data["High"],data["Low"],data["Close"],data["Volume"])
data["AD"] = AD

##ADOSC - Chaikin A/D Oscillator
##real = ADOSC(high, low, close, volume, fastperiod=3, slowperiod=10)
ADOSC = talib.ADOSC(data["High"],data["Low"],data["Close"],data["Volume"], fastperiod=3, slowperiod=10)
data["ADOSC"] = ADOSC

##OBV - On Balance Volume
##real = OBV(close, volume)
OBV = talib.OBV(data["Close"], data["Volume"])
data["OBV"] = OBV

cols3 = [data["AD"]] + [data["ADOSC"]] + [data["OBV"]]



/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/1266846306.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["AD"] = AD
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/1266846306.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["ADOSC"] = ADOSC
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/1266846306.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

In [8]:
##Volatility Indicator

##ATR - Average True Range
## Note: The ATR function has an unstable period.
##real = ATR(high, low, close, timeperiod=14)
for n in range(10,201):
    ATR = talib.ATR(data["High"], data["Low"], data["Close"], timeperiod=n)
    data["ATR_"+str(n)] = ATR


##NATR - Normalized Average True Range
##Note: The NATR function has an unstable period.
##real = NATR(high, low, close, timeperiod=14)
for n in range(10,201):
    NATR = talib.NATR(data["High"], data["Low"], data["Close"], timeperiod=n)
    data["NATR_"+str(n)] = NATR


##TRANGE - True Range
##real = TRANGE(high, low, close)
TRANGE = talib.TRANGE(data["High"], data["Low"], data["Close"])
data["TRANGE"] = TRANGE

cols4 = [data["ATR_"+str(n)] for n in range(10,201)] + [data["NATR_"+str(n)] for n in range(10,201)] + [data["TRANGE"]]



/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/1820558698.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["ATR_"+str(n)] = ATR
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/1820558698.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["ATR_"+str(n)] = ATR
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/1820558698.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance

In [9]:
## Cycle Indicator Functions

## HT_DCPERIOD - Hilbert Transform - Dominant Cycle Period
## Note: The HT_DCPERIOD function has an unstable period.
## real = HT_DCPERIOD(close)
HT_DCPERIOD = talib.HT_DCPERIOD(data["Close"])
data["HT_DCPERIOD"] = HT_DCPERIOD

## HT_DCPHASE - Hilbert Transform - Dominant Cycle Phase
## Note: The HT_DCPHASE function has an unstable period.
## real = HT_DCPHASE(close)
HT_DCPHASE = talib.HT_DCPHASE(data["Close"])
data["HT_DCPHASE"] = HT_DCPHASE


## HT_PHASOR - Hilbert Transform - Phasor Components
## Note: The HT_PHASOR function has an unstable period.
## inphase, quadrature = HT_PHASOR(close)

inphase, quadrature = talib.HT_PHASOR(data["Close"])
data["inphase"] = inphase
data["quadrature"] = quadrature

## HT_SINE - Hilbert Transform - SineWave
## Note: The HT_SINE function has an unstable period.
## sine, leadsine = HT_SINE(close)

sine, leadsine = talib.HT_SINE(data["Close"])
data["sine"] = sine
data["leadsine"] = leadsine

## HT_TRENDMODE - Hilbert Transform - Trend vs Cycle Mode
## Note: The HT_TRENDMODE function has an unstable period.
## integer = HT_TRENDMODE(close)

integer = talib.HT_TRENDMODE(data["Close"])
data["integer"] = integer

cols5 = [data["HT_DCPERIOD"]] + [data["HT_DCPHASE"]] + [data["inphase"]] + [data["quadrature"]] + [
    data["sine"]] + [data["leadsine"]] + [data["integer"]]



/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/608181988.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["HT_DCPERIOD"] = HT_DCPERIOD
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/608181988.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["HT_DCPHASE"] = HT_DCPHASE
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/608181988.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poo

In [10]:
## Price Transform
##AVGPRICE - Average Price
##real = AVGPRICE(open, high, low, close)
AVGPRICE = talib.AVGPRICE(data["Open"],data["High"],data["Low"],data["Close"])
data["AVGPRICE"] = AVGPRICE

##MEDPRICE - Median Price
##real = MEDPRICE(high, low)
MEDPRICE = talib.MEDPRICE(data["High"],data["Low"])
data["MEDPRICE"] = MEDPRICE

##TYPPRICE - Typical Price
##real = TYPPRICE(high, low, close)

TYPPRICE = talib.TYPPRICE(data["High"],data["Low"],data["Close"])
data["TYPPRICE"] = TYPPRICE


##WCLPRICE - Weighted Close Price
##real = WCLPRICE(high, low, close)
WCLPRICE = talib.WCLPRICE(data["High"],data["Low"],data["Close"])
data["WCLPRICE"] = WCLPRICE

cols6 = [data["AVGPRICE"]] + [data["MEDPRICE"]] + [data["TYPPRICE"]] + [data["WCLPRICE"]]


df = pd.concat(cols1 + cols2 + cols3 + cols4 + cols5 + cols6, axis=1)

newframe = df.copy()

    
newframe.tail()


/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/2504204773.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["AVGPRICE"] = AVGPRICE
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/2504204773.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["MEDPRICE"] = MEDPRICE
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/2504204773.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfo

,MA_10,MA_11,MA_12,MA_13,MA_14,MA_15,MA_16,MA_17,MA_18,MA_19,...,HT_DCPHASE,inphase,quadrature,sine,leadsine,integer,AVGPRICE,MEDPRICE,TYPPRICE,WCLPRICE
360,16780.472,16780.463636,16768.234167,16813.706154,16884.492857,16944.326667,16960.611250,16967.912941,16976.911667,16985.043158,...,-12.593551,126.082226,64.052211,-0.218033,0.535922,0,16799.7350,16785.650,16759.046667,16745.7450
361,16761.173,16759.146364,16760.915833,16751.130769,16794.576429,16861.919333,16919.424375,16936.215882,16944.467222,16954.226316,...,7.390637,117.015168,-48.726569,0.128634,0.792190,1,16627.1425,16627.665,16600.406667,16586.7775
362,16780.588,16749.615455,16748.720833,16751.156154,16742.767143,16783.874000,16847.676875,16902.637059,16919.428333,16928.128947,...,26.164476,49.693139,-155.369946,0.440949,0.946449,1,16584.2450,16577.655,16596.450000,16605.8475
363,16751.648,16765.020909,16737.926667,16738.000000,16741.027143,16733.872667,16772.966250,16833.657647,16886.343333,16903.108421,...,35.769322,17.747490,-82.355596,0.584523,0.987051,1,16565.2450,16510.240,16543.276667,16559.7950
364,16721.194,16731.050000,16745.025000,16721.553077,16722.790714,16726.630000,16720.822500,16758.384118,16816.513889,16867.328947,...,55.173804,35.370684,-134.730719,0.820888,0.984276,1,16559.3575,16553.260,16543.863333,16539.1650


In [11]:
## Statistic Functions

##BETA - Beta
##real = BETA(high, low, timeperiod=5)
for n in range(10,201):
    BETA = talib.BETA(data["High"],data["Low"],timeperiod=n)
    data["BETA_"+str(n)] = BETA

##CORREL - Pearson's Correlation Coefficient (r)
##real = CORREL(high, low, timeperiod=30)
for n in range(10,201):
    CORREL = talib.CORREL(data["High"],data["Low"],timeperiod=n)
    data["CORREL_"+str(n)] = CORREL



##LINEARREG - Linear Regression
##real = LINEARREG(close, timeperiod=14)
for n in range (10,201):
    LINEARREG = talib.LINEARREG(data["Close"], timeperiod=n)
    data["LINEARREG_"+str(n)] = LINEARREG
    
##LINEARREG_ANGLE - Linear Regression Angle
##real = LINEARREG_ANGLE(close, timeperiod=14)
for n in range (10,201):
    LINEARREG_ANGLE = talib.LINEARREG_ANGLE(data["Close"], timeperiod=n)
    data["LINEARREG_ANGLE_"+str(n)] = LINEARREG_ANGLE


##LINEARREG_INTERCEPT - Linear Regression Intercept
##real = LINEARREG_INTERCEPT(close, timeperiod=14)
for n in range (10,201):
    LINEARREG_INTERCEPT = talib.LINEARREG_INTERCEPT(data["Close"], timeperiod=n)
    data["LINEARREG_INTERCEPT_"+str(n)] = LINEARREG_INTERCEPT
    
##LINEARREG_SLOPE - Linear Regression Slope
##real = LINEARREG_SLOPE(close, timeperiod=14)
for n in range (10,201):
    LINEARREG_SLOPE = talib.LINEARREG_SLOPE(data["Close"], timeperiod=n)
    data["LINEARREG_SLOPE_"+str(n)] = LINEARREG_SLOPE


##STDDEV - Standard Deviation
##real = STDDEV(close, timeperiod=5, nbdev=1)
for n in range (10,201):
    STDDEV = talib.STDDEV(data["Close"], timeperiod=n, nbdev=1)
    data["STDDEV_"+str(n)] = STDDEV

##TSF - Time Series Forecast
##real = TSF(close, timeperiod=14)
for n in range (10,201):
    TSF = talib.TSF(data["Close"], timeperiod=n)
    data["TSF_"+str(n)] = TSF

##VAR - Variance
##real = VAR(close, timeperiod=5, nbdev=1)
for n in range (10,201):
    VAR = talib.VAR(data["Close"], timeperiod=n, nbdev=1)
    data["VAR_"+str(n)] = VAR


cols7 = [data["BETA_"+str(n)] for n in range (10,201)] + [data["CORREL_"+str(n)] for n in range (10,201)] + [
    data["LINEARREG_"+str(n)] for n in range (10,201)] +[data["LINEARREG_ANGLE_"+str(n)]for n in range (10,201)] +[
    data["LINEARREG_INTERCEPT_"+str(n)] for n in range (10,201)] +[data["LINEARREG_SLOPE_"+str(n)] for n in range (10,201)] + [
    data["STDDEV_"+str(n)] for n in range (10,201)] + [data["TSF_"+str(n)]for n in range (10,201)] + [data["VAR_"+str(n)] for n in range (10,201)
]




/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/3498125634.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["BETA_"+str(n)] = BETA
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/3498125634.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["BETA_"+str(n)] = BETA
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/3498125634.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

In [12]:
##Math Transform Functions

##ACOS - Vector Trigonometric ACos
##real = ACOS(close)
ACOS = talib.ACOS(data["Close"])
data["ACOS"] = ACOS

##ASIN - Vector Trigonometric ASin
##real = ASIN(close)
ASIN = talib.ASIN(data["Close"])
data["ASIN"] = ASIN

##ATAN - Vector Trigonometric ATan
##real = ATAN(close)
ATAN = talib.ATAN(data["Close"])
data["ATAN"] = ATAN

##CEIL - Vector Ceil
##real = CEIL(close)
CEIL = talib.CEIL(data["Close"])
data["CEIL"] = CEIL

##COS - Vector Trigonometric Cos
##real = COS(close)
COS = talib.COS(data["Close"])
data["COS"] = COS

##COSH - Vector Trigonometric Cosh
##real = COSH(close)
COSH = talib.COSH(data["Close"])
data["ASIN"] = COSH

##EXP - Vector Arithmetic Exp
##real = EXP(close)
EXP = talib.EXP(data["Close"])
data["EXP"] = EXP

##FLOOR - Vector Floor
##real = FLOOR(close)
FLOOR = talib.FLOOR(data["Close"])
data["FLOOR"] = FLOOR

##LN - Vector Log Natural
##real = LN(close)
LN = talib.LN(data["Close"])
data["LN"] = LN

##LOG10 - Vector Log10
##real = LOG10(close)
LOG10 = talib.LOG10(data["Close"])
data["LOG10"] = LOG10

##SIN - Vector Trigonometric Sin
##real = SIN(close)
SIN = talib.SIN(data["Close"])
data["SIN"] = SIN


##SINH - Vector Trigonometric Sinh
##real = SINH(close)
SINH = talib.SINH(data["Close"])
data["SINH"] = SINH


##SQRT - Vector Square Root
##real = SQRT(close)
SQRT = talib.SQRT(data["Close"])
data["SQRT"] = SQRT

##TAN - Vector Trigonometric Tan
##real = TAN(close)
TAN = talib.TAN(data["Close"])
data["TAN"] = TAN


##TANH - Vector Trigonometric Tanh
##real = TANH(close)
TANH = talib.TANH(data["Close"])
data["TANH"] = TANH

cols8 = [data["ACOS"]]+[data["ASIN"]]+[data["ATAN"]]+[data["CEIL"]]+[data["COS"]]+[data["ASIN"]]+[data["EXP"]]+[
    data["FLOOR"]]+[data["LN"]]+[data["LOG10"]]+[data["SIN"]]+[data["SINH"]]+[data["SQRT"]]+[data["TAN"]]+[data["TANH"]]


df = pd.concat(cols1 + cols2 + cols3 + cols4 + cols5 + cols6 + cols7 + cols8, axis=1)

newframe = df.copy()

    
newframe.tail()

/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/3717088968.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["ACOS"] = ACOS
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/3717088968.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["ASIN"] = ASIN
/var/folders/yn/_rglt16j18j_1p61vqk0b68w0000gn/T/ipykernel_75556/3717088968.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Conside

,MA_10,MA_11,MA_12,MA_13,MA_14,MA_15,MA_16,MA_17,MA_18,MA_19,...,ASIN,EXP,FLOOR,LN,LOG10,SIN,SINH,SQRT,TAN,TANH
360,16780.472,16780.463636,16768.234167,16813.706154,16884.492857,16944.326667,16960.611250,16967.912941,16976.911667,16985.043158,...,inf,inf,16705.0,9.723514,4.222868,-0.912654,inf,129.251073,-2.232891,1.0
361,16761.173,16759.146364,16760.915833,16751.130769,16794.576429,16861.919333,16919.424375,16936.215882,16944.467222,16954.226316,...,inf,inf,16545.0,9.713893,4.218690,0.769786,inf,128.630828,-1.205992,1.0
362,16780.588,16749.615455,16748.720833,16751.156154,16742.767143,16783.874000,16847.676875,16902.637059,16919.428333,16928.128947,...,inf,inf,16634.0,9.719206,4.220998,0.638926,inf,128.973020,-0.830562,1.0
363,16751.648,16765.020909,16737.926667,16738.000000,16741.027143,16733.872667,16772.966250,16833.657647,16886.343333,16903.108421,...,inf,inf,16609.0,9.717721,4.220353,0.247752,inf,128.877267,-0.255725,1.0
364,16721.194,16731.050000,16745.025000,16721.553077,16722.790714,16726.630000,16720.822500,16758.384118,16816.513889,16867.328947,...,inf,inf,16525.0,9.712634,4.218143,0.288483,inf,128.549874,0.301292,1.0


In [13]:
import psycopg2

# Connect to the database
conn = psycopg2.connect(
    host="localhost",
    database="btcdatabase",
    user="Chris",
    password="1234"
)

# Write the DataFrame to the database
df.to_sql("btc_indicators", conn, if_exists="replace", index=False)

# Commit the changes
conn.commit()

# Close the connection
conn.close()


OperationalError: could not connect to server: Connection refused
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?


In [ ]:
# Grades
## investor-grades
import pandas as pd
import requests
from pandas import json_normalize
from urllib.parse import urlencode


API_key = 'tm-641f8fcd-91a3-474a-8e72-96334992faf2'

def tm_API(endpoint, payload):
	if payload:
		url = 'https://alpha.data-api.tokenmetrics.com/v1/' + endpoint + '?' + urlencode(payload)
	else:
		url = 'https://alpha.data-api.tokenmetrics.com/v1/' + endpoint 
	headers = {'Accept': 'application/json', 'Content-Type': 'application/json', 'api_key': API_key}
	response = requests.get(url, headers=headers)
	return response.json()
	

endpoint = 'investor-grades'
params = {
	'tokens': '3375',
	'startDate': '2022-01-01T00:00:00.000Z',
	'endDate': '2022-12-31T23:59:59.000Z',
	'limit' : 1000
}
response = tm_API(endpoint,params)
dt_1 = json_normalize(response['data'])
dt_1_col = ['Date','SYMBOL','EPOCH','TM_INVESTOR_GRADE','VALUATION_GRADE','TECHNOLOGY_GRADE']

print(pd.DataFrame(dt_1, columns = dt_1_col).tail())

"""
   TOKEN_ID      NAME SYMBOL  ...  TM_INVESTOR_GRADE       EPOCH        DATE
0      3306  Ethereum    ETH  ...          82.061179  1669507200  2022-11-27
1      3375   Bitcoin    BTC  ...          71.465773  1669507200  2022-11-27
2      3306  Ethereum    ETH  ...          80.069432  1669334400  2022-11-25
3      3375   Bitcoin    BTC  ...          71.685027  1669334400  2022-11-25
"""

     Date SYMBOL       EPOCH  TM_INVESTOR_GRADE VALUATION_GRADE  \
318   NaN    BTC  1644537600                NaN            None   
319   NaN    BTC  1644451200                NaN            None   
320   NaN    BTC  1644364800                NaN            None   
321   NaN    BTC  1644278400                NaN            None   
322   NaN    BTC  1644192000                NaN            None   

     TECHNOLOGY_GRADE  
318         97.935689  
319         97.935689  
320         97.935689  
321         97.935689  
322         97.935689  


'\n   TOKEN_ID      NAME SYMBOL  ...  TM_INVESTOR_GRADE       EPOCH        DATE\n0      3306  Ethereum    ETH  ...          82.061179  1669507200  2022-11-27\n1      3375   Bitcoin    BTC  ...          71.465773  1669507200  2022-11-27\n2      3306  Ethereum    ETH  ...          80.069432  1669334400  2022-11-25\n3      3375   Bitcoin    BTC  ...          71.685027  1669334400  2022-11-25\n'

In [ ]:
## Modify the timestemp and date



In [ ]:
## Find data Function 
## Type the time and indicator's name, the data will jump out
## Need add date
result = data.at[data[data['Open time'] == 1672444800000].index[0],'MEDPRICE']
print(result)


16553.260000000002


In [ ]:
class My_indicator:
    def __init__(self, time_period, date):
        self.time_period = time_period
        self.date = date
        git demofff

    def _check_param(self, param):
        for p in param:
            if p < 0:
                raise ValueError('param must be positive')
        self.result = talib.EMA(data['Close'], timeperiod)
    
#######################
## To obtain all the param name

param_list = db.get_params()
''''''
yes
{
    "EMA": {
        "param1": "timeperiod",
        "param2": "date",
    }
}


NameError: name 'EMA' is not defined